## Loading packages

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import seaborn as sns
from google.colab import files
!pip install xlrd
import xlrd
from sklearn.preprocessing import LabelBinarizer, LabelEncoder 
from sklearn.model_selection import train_test_split

    100% |████████████████████████████████| 112kB 4.3MB/s 


## Titanic dataset

In [0]:
data = files.urllib.request.urlretrieve('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls', filename=None)[0]
df = pd.read_excel(data)

### Data preprocessing

Firstly, let's get rid of features that have too many nans, as was decided in EDA.

In [0]:
df.drop(columns=['cabin', 'boat', 'body', 'home.dest', 'name'], inplace=True)

In [0]:
for column in df.columns:
  if (df[column].dtypes in ['float64', 'int64']) and sum(df[column].isnull()):
    df[column].fillna(df[column].median(), inplace=True)  
  if (df[column].dtypes == 'object') and sum(df[column].isnull()):
    df[column].fillna(df[column].mode()[0], inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
pclass      1309 non-null int64
survived    1309 non-null int64
sex         1309 non-null object
age         1309 non-null float64
sibsp       1309 non-null int64
parch       1309 non-null int64
ticket      1309 non-null object
fare        1309 non-null float64
embarked    1309 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 92.1+ KB


In [6]:
df.ticket = df.ticket.str.replace(' \d+', '').str.replace('[0-9][0-9]+', '').replace([np.nan, ''], 'simple')
df.ticket.unique()

array(['simple', u'PC', u'W.E.P.', u'WE/P', u'F.C.', u'P/PP', u'C.A.',
       u'C.A./SOTON', u'S.O.P.', u'F.C.C.', u'SC/AH', u'W./C.', u'S.O.C.',
       u'SC/PARIS', u'S.W./PP', u'W/C', u'SCO/W', u'SO/C', u'SC/AH Basle',
       u'SC/Paris', u'SC', u'S.O./P.P.', u'S.C./PARIS', u'SC/A.3',
       u'SW/PP', u'CA', u'SOTON/O2', u'C', u'SOTON/O.Q.', u'A/4', u'PP',
       u'A/5', u'A./5.', u'A/5.', u'SOTON/OQ', u'S.P.', u'SC/A4',
       u'AQ/3.', u'STON/O2.', u'A..', u'LINE', u'STON/O.', u'AQ/4',
       u'A/S', u'A.5.', u'A4.', u'A/4.', u'Fa', u'S.C./A.4.', u'LP',
       u'CA.', u'STON/OQ.'], dtype=object)

In [0]:
binarizer = LabelBinarizer()
encoder = LabelEncoder()

In [0]:
df.sex = binarizer.fit_transform(df.sex)
df.ticket = encoder.fit_transform(df.ticket)
df = pd.get_dummies(df)

In [11]:
df.head()

,pclass,survived,sex,age,sibsp,parch,ticket,fare,embarked_C,embarked_Q,embarked_S
0,1,1,0,29.0000,0,0,51,211.3375,0,0,1
1,1,1,1,0.9167,1,2,51,151.5500,0,0,1
2,1,0,0,2.0000,1,2,51,151.5500,0,0,1
3,1,0,1,30.0000,1,2,51,151.5500,0,0,1
4,1,0,0,25.0000,1,2,51,151.5500,0,0,1


### Train-test split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='survived'), df.survived, test_size=0.3, random_state=42)

### Neural Network building

In [15]:
X_train.shape

(916, 10)

In [0]:
tf.placeholder(tf.float32, [None, 10])
